In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
from groq import Groq
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import json



In [25]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rmp-rag", dimension = 384, metric = "cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [22]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# Define the index name
index_name = "rmp-rag"
index=pc.Index("rmp-rag")


In [12]:
import json
data=json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Introduction to Psychology',
  'stars': 5,
  'review': 'Dr. Johnson is an amazing professor! Her lectures are engaging, and she genuinely cares about her students.'},
 {'professor': 'Professor Michael Lee',
  'subject': 'Calculus I',
  'stars': 4,
  'review': 'Great professor, but the material can be tough. He explains concepts well, though you need to put in the effort.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Organic Chemistry',
  'stars': 3,
  'review': 'Dr. Thompson knows her stuff, but her exams are really challenging. Be prepared to study hard.'},
 {'professor': 'Professor James Harris',
  'subject': 'World History',
  'stars': 2,
  'review': 'Lectures are a bit dry, and the grading is tough. Not my favorite class.'},
 {'professor': 'Dr. Laura Martinez',
  'subject': 'Introduction to Sociology',
  'stars': 5,
  'review': 'Dr. Martinez is incredible! She makes complex topics easy to understand and always encourages dis

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings
processed_data = []
# Load the pretrained model
model= HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Create embeddings (numerical sematic relationship of words)

for review in data["reviews"]:
    embedding = model.embed_query(review['review'])


    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata":{
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })



[{'values': [-0.03828218951821327, -0.05588771402835846, 0.00939775537699461, 0.019271988421678543, -0.022335313260555267, -0.006937576923519373, 0.0021682081278413534, 0.025159651413559914, -0.007017605006694794, -0.013651794753968716, -0.04879738762974739, 0.12544138729572296, -0.037571147084236145, 0.021840890869498253, -0.07378734648227692, 0.037701964378356934, 0.010066073387861252, -0.0053136167116463184, -0.020637253299355507, -0.03003130853176117, 0.05055741220712662, 0.026795707643032074, 0.08230900764465332, -0.00249143922701478, -0.025874856859445572, -0.004667050205171108, 0.02155626006424427, -0.07149616628885269, 0.017512504011392593, -0.01155536063015461, -0.06850511580705643, 0.03933820500969887, -0.07149490714073181, 0.013472849503159523, -0.003251418936997652, 0.011962082237005234, 0.008243412710726261, 0.07894453406333923, 0.038007721304893494, -0.010103700682520866, -0.04797804355621338, 0.002358752768486738, 0.09318368136882782, 0.010730785317718983, -0.03392025828

In [15]:

processed_data[0]

{'values': [-0.03828218951821327,
  -0.05588771402835846,
  0.00939775537699461,
  0.019271988421678543,
  -0.022335313260555267,
  -0.006937576923519373,
  0.0021682081278413534,
  0.025159651413559914,
  -0.007017605006694794,
  -0.013651794753968716,
  -0.04879738762974739,
  0.12544138729572296,
  -0.037571147084236145,
  0.021840890869498253,
  -0.07378734648227692,
  0.037701964378356934,
  0.010066073387861252,
  -0.0053136167116463184,
  -0.020637253299355507,
  -0.03003130853176117,
  0.05055741220712662,
  0.026795707643032074,
  0.08230900764465332,
  -0.00249143922701478,
  -0.025874856859445572,
  -0.004667050205171108,
  0.02155626006424427,
  -0.07149616628885269,
  0.017512504011392593,
  -0.01155536063015461,
  -0.06850511580705643,
  0.03933820500969887,
  -0.07149490714073181,
  0.013472849503159523,
  -0.003251418936997652,
  0.011962082237005234,
  0.008243412710726261,
  0.07894453406333923,
  0.038007721304893494,
  -0.010103700682520866,
  -0.04797804355621338,


In [26]:
index = pc.Index('rmp-rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [27]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}

In [49]:
import requests
page = requests.get("https://www.ratemyprofessors.com/professor/635790")
page

<Response [200]>

In [50]:
page.content

b'\n    <!DOCTYPE html>\n    <!-- SSR -->\n    <html >\n      <head>\n        <meta name="viewport" content="width=device-width, initial-scale=1" />\n        <meta name="theme-color" content="#000000" />\n        <meta name="thumbnail" content="https://www.ratemyprofessors.com/build/thumbnail.svg" />\n\n        \n\n        <link rel="manifest" href="/build/manifest.json">\n        <link rel="stylesheet" type="text/css" href="/static/css/main.1773c5b7.css">\n    \n        <!-- Nobid Prebid Wrapper -->\n        <script async id="nobid-wrapper" src="https://public.servenobid.com/partner/66897/56514/wrapup_1.0.2.js"></script>\n        <script>\n          window.wrapup = window.wrapup || { cmd: [] };\n        </script>\n        \n        <link data-react-helmet="true" rel="icon" href="/favicons/favicon-16.png" sizes/><link data-react-helmet="true" rel="icon" href="/favicons/favicon-32.png" sizes="32\xc3\x9732"/><link data-react-helmet="true" rel="apple-touch-icon" href="/favicons/favicon-57

In [51]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [52]:
print(soup.prettify())

<!DOCTYPE html>
<!-- SSR -->
<html>
 <head>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="https://www.ratemyprofessors.com/build/thumbnail.svg" name="thumbnail"/>
  <link href="/build/manifest.json" rel="manifest"/>
  <link href="/static/css/main.1773c5b7.css" rel="stylesheet" type="text/css"/>
  <!-- Nobid Prebid Wrapper -->
  <script async="" id="nobid-wrapper" src="https://public.servenobid.com/partner/66897/56514/wrapup_1.0.2.js">
  </script>
  <script>
   window.wrapup = window.wrapup || { cmd: [] };
  </script>
  <link data-react-helmet="true" href="/favicons/favicon-16.png" rel="icon" sizes="">
   <link data-react-helmet="true" href="/favicons/favicon-32.png" rel="icon" sizes="32×32">
    <link data-react-helmet="true" href="/favicons/favicon-57.png" rel="apple-touch-icon" sizes=""/>
    <link data-react-helmet="true" href="/favicons/favicon-48.png" rel="apple-touch-icon" sizes="48×48"/>
  

In [34]:
# Select all elements at the top level of the page
list(soup.children)

['\n',
 'html',
 '\n',
 ' SSR ',
 '\n',
 <html>
 <head>
 <meta content="width=device-width, initial-scale=1" name="viewport"/>
 <meta content="#000000" name="theme-color"/>
 <meta content="https://www.ratemyprofessors.com/build/thumbnail.svg" name="thumbnail"/>
 <link href="/build/manifest.json" rel="manifest"/>
 <link href="/static/css/main.1773c5b7.css" rel="stylesheet" type="text/css"/>
 <!-- Nobid Prebid Wrapper -->
 <script async="" id="nobid-wrapper" src="https://public.servenobid.com/partner/66897/56514/wrapup_1.0.2.js"></script>
 <script>
           window.wrapup = window.wrapup || { cmd: [] };
         </script>
 <link data-react-helmet="true" href="/favicons/favicon-16.png" rel="icon" sizes=""><link data-react-helmet="true" href="/favicons/favicon-32.png" rel="icon" sizes="32×32"><link data-react-helmet="true" href="/favicons/favicon-57.png" rel="apple-touch-icon" sizes=""/><link data-react-helmet="true" href="/favicons/favicon-48.png" rel="apple-touch-icon" sizes="48×48"/><l

In [35]:
# Find the types of the top elements 
[type(item) for item in list(soup.children)]

[bs4.element.NavigableString,
 bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Comment,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [53]:
# Select the tag and its children 
html = list(soup.children)[5]
list(html.children)

['\n',
 <head>
 <meta content="width=device-width, initial-scale=1" name="viewport"/>
 <meta content="#000000" name="theme-color"/>
 <meta content="https://www.ratemyprofessors.com/build/thumbnail.svg" name="thumbnail"/>
 <link href="/build/manifest.json" rel="manifest"/>
 <link href="/static/css/main.1773c5b7.css" rel="stylesheet" type="text/css"/>
 <!-- Nobid Prebid Wrapper -->
 <script async="" id="nobid-wrapper" src="https://public.servenobid.com/partner/66897/56514/wrapup_1.0.2.js"></script>
 <script>
           window.wrapup = window.wrapup || { cmd: [] };
         </script>
 <link data-react-helmet="true" href="/favicons/favicon-16.png" rel="icon" sizes=""><link data-react-helmet="true" href="/favicons/favicon-32.png" rel="icon" sizes="32×32"><link data-react-helmet="true" href="/favicons/favicon-57.png" rel="apple-touch-icon" sizes=""/><link data-react-helmet="true" href="/favicons/favicon-48.png" rel="apple-touch-icon" sizes="48×48"/><link data-react-helmet="true" href="/favic

In [41]:
body = list(html.children)[3]

In [42]:
list(body.children)

['\n',
 ' Google Tag Manager (noscript) ',
 '\n',
 <noscript><iframe height="0" src="https://www.googletagmanager.com/ns.html?id=GTM-5HVQ56V" style="display:none;visibility:hidden" width="0"></iframe></noscript>,
 '\n',
 ' End Google Tag Manager (noscript) ',
 '\n',
 <div id="root"><div class="App__StyledApp-aq7j9t-0 kaBGnN"><div class="App__Body-aq7j9t-1 bhyGpW"><div class="HeaderContainer__StyledHeaderContainer-sc-1eya4wu-0 ducGvC" data-testid="StyledHeaderContainer"><div class="Header__HeaderBannerWrapper-sc-1oduwjk-5 enzK"><header class="Header__StyledHeader-sc-1oduwjk-1 hNQJBj" role="banner"><div class="Toastify"></div><div class="Header__HeaderWrapper-sc-1oduwjk-4 eLUjyu"><div class="Header__StyledDummyElement-sc-1oduwjk-3 kjTgMJ"></div><div class="HeaderLogo__StyledHeaderLogo-sc-1ehuz2f-2 vWase"><div class="MobileMenu__MobileMenuWrapper-mf2apz-0 ecMUyP"><button class="MobileMenu__MenuButton-mf2apz-2 cdrVhb" title="Mobile Menu Button"></button></div><a href="/"><img alt="Logo" cl

In [64]:
name = soup.find(class_="NameTitle__Name-dowf0z-0 cfjPUG").getText()
print(name)
rating = soup.find(class_="RatingValue__Numerator-qw8sqy-2 liyUjw").getText()
print(rating)
subject = soup.find(class_="TeacherDepartment__StyledDepartmentLink-fl79e8-0 iMmVHb").getText()
print(subject)
ratings = soup.select(".Comments__StyledComments-dzzyvm-0 gRjWel")
reviews=[rev.getText() for rev in ratings]
print(ratings)



A Pierre 
2.8
Counseling department
[]
